```
This software is a part of GPU Ocean.

Copyright (C) 2019  SINTEF Digital

In this notebook we carry out prototyping for developing a new 
ensemble class that can be used for reading observations from file
and still work in the current Data Assimilation structure.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Development of a new ensemble class based on files

In this notebook we carry out prototyping for developing a new 
ensemble class that can be used for reading observations from file
and still work in the current Data Assimilation structure.


## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc

import pycuda.driver as cuda
import os
import sys
import datetime

from importlib import reload
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import IPythonMagic, CDKLM16, EnsembleFromFiles

from SWESimulators import BaseOceanStateEnsemble, SimReader, Observation
from SWESimulators import DataAssimilationUtils as dautils


In [ ]:
%cuda_context_handler gpu_ctx

In [ ]:
#Create output directory for images
#imgdir = 'double_jet'
#filename_prefix = imgdir + "/" + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S") + "_"
#os.makedirs(imgdir, exist_ok=True)
#print("Saving images to " + imgdir)

### Define functions for plotting

In [ ]:
def imshow(im, interpolation="None", title=None, figsize=(4,4), interior=False):
    fig = plt.figure(figsize=figsize)
    
    if interior:
        im = plt.imshow(im[2:-2,2:-2], interpolation=interpolation, origin='lower')
    else:
        im = plt.imshow(im, interpolation=interpolation, origin='lower')
    
    plt.colorbar()
    if title is not None:
        plt.title(title)
        
def imshow3(eta, hu, hv, interpolation="None", title=None, figsize=(12,3), 
            interior=False, color_bar_from_zero=False):
    fig, axs = plt.subplots(1,3, figsize=figsize)
    
    eta_max = np.max(np.abs(eta))
    huv_max = max(np.max(np.abs(hu)), np.max(np.abs(hv)))
    eta_min = -eta_max
    huv_min = -huv_max
    if color_bar_from_zero:
        eta_min, huv_min = 0, 0
    
    if interior:
        eta_im = axs[0].imshow(eta[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=eta_min, vmax=eta_max)
    else:
        eta_im = axs[0].imshow(eta, interpolation=interpolation, origin='lower', vmin=eta_min, vmax=eta_max)
    axs[0].set_title("$\eta$")
    plt.colorbar(eta_im, ax=axs[0])
    
    if interior:
        hu_im = axs[1].imshow(hu[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=huv_min, vmax=huv_max)
    else:
        hu_im = axs[1].imshow(hu, interpolation=interpolation, origin='lower', vmin=huv_min, vmax=huv_max)
    axs[1].set_title("$hu$")
    plt.colorbar(hu_im, ax=axs[1])

    if interior:
        hv_im = axs[2].imshow(hv[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=huv_min, vmax=huv_max)
    else:
        hv_im = axs[2].imshow(hv, interpolation=interpolation, origin='lower', vmin=huv_min, vmax=huv_max)
    axs[2].set_title("$hv$")
    plt.colorbar(hv_im, ax=axs[2])

    if title is not None:
        plt.suptitle(title)
    plt.tight_layout()
    
def plotDrifters(observations, sim_reader, t, drifter_set=[]):
    drifter_positions = observations.get_drifter_position(t)

    fig = plt.figure(figsize=(7,7))
    ax = plt.subplot(111)
    nx, ny = sim_reader.get('nx'), sim_reader.get('ny')
    dx, dy = sim_reader.get('dx'), sim_reader.get('dy')

    emptyData =np.ma.masked_where(np.zeros((ny,nx)) > 1, np.zeros((ny,nx)))
    ax.imshow(emptyData, origin="lower", extent=[0, nx*dx, 0, ny*dy], cmap='binary')

    for i in range(drifter_positions.shape[0]):
        color = 'xkcd:pale cyan'
        if i in drifter_set:
            color = 'xkcd:tomato red'
        circ_end = matplotlib.patches.Circle((drifter_positions[i,0], drifter_positions[i,1]),
                                             3000, fill=True, zorder=10, color=color)
        ax.add_patch(circ_end)

    
def days_to_sec(days):
    return days*24*60*60

def truth_time_step(t):
    t = t - days_to_sec(3)
    return int(t/(60*60))

In [82]:
test = np.linalg.inv(np.diag([3,445]))
print(test)
print(type(test))
print(test.dtype)
test = test.astype(np.float32)
print(test.dtype)



[[0.33333333 0.        ]
 [0.         0.00224719]]
<class 'numpy.ndarray'>
float64
float32


# The new class

We base in on the OceanStateEnsemble.

In [58]:
if 'ensemble' in globals():
    ensemble.cleanUp()
    del ensemble
    
reload(BaseOceanStateEnsemble)
reload(Observation)
reload(EnsembleFromFiles)

ensemble_init_path = os.path.abspath('double_jet_ensemble_init/')
truth_path = os.path.abspath('double_jet_truth/')
#ensemble = EnsembleFromFile(

print(os.path.isdir(ensemble_init_path))
print(os.path.isdir(truth_path))

ensemble_nc_gen = (os.path.join(ensemble_init_path, file)  for file in os.listdir(ensemble_init_path) if file.endswith('.nc'))
ensemble_nc_files = list(ensemble_nc_gen)
print(type(ensemble_nc_files))
print(len(ensemble_nc_files))
print(ensemble_nc_files[10])
print(type(ensemble_nc_files[10]))
print()
print()

ensemble = EnsembleFromFiles.EnsembleFromFiles(gpu_ctx, 10, ensemble_init_path, truth_path)

Exception ignored in: <bound method BaseOceanStateEnsemble.__del__ of <SWESimulators.EnsembleFromFiles.EnsembleFromFiles object at 0x7ff4b92ece48>>
Traceback (most recent call last):
  File "/home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/SWESimulators/BaseOceanStateEnsemble.py", line 234, in __del__
    self.cleanUp()
  File "/home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/SWESimulators/EnsembleFromFiles.py", line 161, in cleanUp
    self.true_state = None
TypeError: super(type, obj): obj must be an instance or subtype of type


True
True
<class 'list'>
100
/home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/demos/DAPaper/double_jet_ensemble_init/double_jet_case_55.nc
<class 'str'>


Welcome to the EnsembleFromFile
Ensemble directory:  /home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/demos/DAPaper/double_jet_ensemble_init
True state directory:  /home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/demos/DAPaper/double_jet_truth
self.write_netcdf_directory:  /home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/demos/DAPaper/ensemble_result_2019_04_24-11_37_13/
/home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/demos/DAPaper/ensemble_result_2019_04_24-11_37_13/ensemble_member_0000.nc
/home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/demos/DAPaper/ensemble_result_2019_04_24-11_37_13/ensemble_member_0001.nc
/home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/demos/DAPaper/ensemble_result_2019_04_24-11_37_13/ensemble_member_0002.nc
/hom

In [46]:
ensemble.observations.get_num_drifters()

64

In [57]:
print(ensemble.write_netcdf_directory)
ensemble.ensemble_init_nc_files[0].split('/')[-1]

/home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/demos/DAPaper/ensemble_result_2019_04_24-11_32_12/


'double_jet_case_31.nc'

In [63]:
if 'numParticles' in ensemble.__getattribute__():
    print("found!")
else:
    print("not found")
ensemble.numParticles

not found


10

In [65]:
ensemble.__getattribute__('numParticles'), ensemble.__getattribute__('bgawbga')

Exception caught: Resetting to CUDA context gpu_ctx
Traceback (most recent call last):
  File "/home/havahol/miniconda3/envs/gpuocean/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-65-0ecf6d13f9d5>", line 1, in <module>
    ensemble.__getattribute__('numParticles'), ensemble.__getattribute__('bgawbga')
AttributeError: 'EnsembleFromFiles' object has no attribute 'bgawbga'


AttributeError: 'EnsembleFromFiles' object has no attribute 'bgawbga'

In [ ]:
ensemble.plotEnsemble(plotVelocityField=False)
print("hei")